In [39]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt
from ipyleaflet import Map, GeoJSON, GeoData, basemaps, LayersControl
from shapely.geometry import Point, Polygon, LineString
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
import rasterio
import rasterio.mask
from rasterio.plot import reshape_as_image 
from rasterio.plot import reshape_as_raster
from shapely.ops import unary_union

In [3]:
# Opening JSON file
f = open('Rotterdam_data/historic_towages.json')
 
# returns JSON object as a dictionary
tow_data = json.load(f)
 
# Closing file
f.close()

In [31]:
# Create empty lists to store tugs data and their original JSON index
tugs_data = []
original_index = []

# Extract tugs data into a separate list and store their original index
for i, item in enumerate(tow_data):
    for tug in item['tugs']:
        tugs_data.append(tug)
        original_index.append(i)

# Create a DataFrame for tugs data with the original index
tugs_df = pd.DataFrame(tugs_data)
tugs_df['original_index'] = original_index  # Add original index as a new column


# Create a DataFrame for the rest of the data
vessel_data = [{'from': item['from'],
               'to': item['to'],
               'vessel': item['vessel'],
               'type': item['type'],
               'additional_data': item['additional_data']}
              for item in tow_data]

vessel_df = pd.DataFrame(vessel_data)

# Display the DataFrames
tugs_df

,imo,mmsi,name,from,to,from_location,to_location,from_berth,to_berth,from_haven,to_haven,original_index
0,9695509,244900124,VB MARS,2022-05-31 22:49:33+00:00,2022-06-01 00:10:13+00:00,"[4.06852, 51.98272]","[4.05362, 51.9375]",None,MISSISSIPPIH EMO MH 4,None,MISS,0
1,9816658,248352000,ROTTERDAM,2022-05-31 22:49:32+00:00,2022-06-01 00:20:21+00:00,"[4.06852, 51.98272]","[4.05363, 51.93748]",None,MISSISSIPPIH EMO MH 4,None,MISS,0
2,9816359,248753000,BEAGLE,2022-05-31 22:54:16+00:00,2022-06-01 00:35:37+00:00,"[4.08358, 51.97713]","[4.05365, 51.93745]",None,MISSISSIPPIH EMO MH 4,None,MISS,0
3,9489936,244697000,VB KRACHT,2022-05-31 23:29:57+00:00,2022-06-01 00:30:17+00:00,"[4.0516, 51.94783]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1
4,9474905,244791000,RT ROB,2022-05-31 23:20:43+00:00,2022-06-01 00:30:17+00:00,"[4.05166, 51.94792]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...
42227,9060704,245906000,THAMESBANK,2023-05-31 22:13:18+00:00,2023-05-31 22:59:18+00:00,"[4.44552, 51.8936]","[4.41163, 51.90059]",WAALH HBR B 23,None,WAAL,None,24491
42228,9507063,245932000,VB GINGER,2023-05-31 23:04:20+00:00,2023-05-31 23:44:50+00:00,"[4.1589, 51.94893]","[4.1935, 51.93023]",None,CALANDK ETT EAST,None,CKVTTI,24492
42229,9060704,245906000,THAMESBANK,2023-05-31 23:09:28+00:00,2023-05-31 23:44:56+00:00,"[4.39814, 51.87972]","[4.38662, 51.89737]",ALEXANDERH ZZ 2,None,PWAH,None,24493
42230,9120190,205233000,UNION 11,2023-05-31 23:04:17+00:00,2023-05-31 23:49:58+00:00,"[4.39814, 51.87972]","[4.37126, 51.89949]",ALEXANDERH ZZ 2,None,PWAH,None,24493


In [32]:
tugs_df = tugs_df.merge(vessel_df[['type']], left_on='original_index', right_index=True)
tugs_df

,imo,mmsi,name,from,to,from_location,to_location,from_berth,to_berth,from_haven,to_haven,original_index,type
0,9695509,244900124,VB MARS,2022-05-31 22:49:33+00:00,2022-06-01 00:10:13+00:00,"[4.06852, 51.98272]","[4.05362, 51.9375]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming
1,9816658,248352000,ROTTERDAM,2022-05-31 22:49:32+00:00,2022-06-01 00:20:21+00:00,"[4.06852, 51.98272]","[4.05363, 51.93748]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming
2,9816359,248753000,BEAGLE,2022-05-31 22:54:16+00:00,2022-06-01 00:35:37+00:00,"[4.08358, 51.97713]","[4.05365, 51.93745]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming
3,9489936,244697000,VB KRACHT,2022-05-31 23:29:57+00:00,2022-06-01 00:30:17+00:00,"[4.0516, 51.94783]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1,leaving
4,9474905,244791000,RT ROB,2022-05-31 23:20:43+00:00,2022-06-01 00:30:17+00:00,"[4.05166, 51.94792]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1,leaving
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42227,9060704,245906000,THAMESBANK,2023-05-31 22:13:18+00:00,2023-05-31 22:59:18+00:00,"[4.44552, 51.8936]","[4.41163, 51.90059]",WAALH HBR B 23,None,WAAL,None,24491,leaving
42228,9507063,245932000,VB GINGER,2023-05-31 23:04:20+00:00,2023-05-31 23:44:50+00:00,"[4.1589, 51.94893]","[4.1935, 51.93023]",None,CALANDK ETT EAST,None,CKVTTI,24492,incoming
42229,9060704,245906000,THAMESBANK,2023-05-31 23:09:28+00:00,2023-05-31 23:44:56+00:00,"[4.39814, 51.87972]","[4.38662, 51.89737]",ALEXANDERH ZZ 2,None,PWAH,None,24493,leaving
42230,9120190,205233000,UNION 11,2023-05-31 23:04:17+00:00,2023-05-31 23:49:58+00:00,"[4.39814, 51.87972]","[4.37126, 51.89949]",ALEXANDERH ZZ 2,None,PWAH,None,24493,leaving


In [42]:
tugs_incoming = tugs_df[tugs_df['type']=='incoming']
tugs_incoming = tugs_incoming[['from',	'to',	'from_location' ,'to_haven', 'original_index']]

# Create a Shapely Point geometry from the "from_location" coordinates
tugs_incoming['geometry'] = tugs_incoming['from_location'].apply(lambda coord: Point(coord))

# Convert the DataFrame to a GeoDataFrame
tugs_incoming_gdf = gpd.GeoDataFrame(tugs_incoming, geometry='geometry')
tugs_incoming_gdf = tugs_incoming_gdf.set_crs("EPSG:4326")

In [47]:
tugs_incoming_gdf

,from,to,from_location,to_haven,original_index,geometry
0,2022-05-31 22:49:33+00:00,2022-06-01 00:10:13+00:00,"[4.06852, 51.98272]",MISS,0,POINT (4.06852 51.98272)
1,2022-05-31 22:49:32+00:00,2022-06-01 00:20:21+00:00,"[4.06852, 51.98272]",MISS,0,POINT (4.06852 51.98272)
2,2022-05-31 22:54:16+00:00,2022-06-01 00:35:37+00:00,"[4.08358, 51.97713]",MISS,0,POINT (4.08358 51.97713)
5,2022-05-31 23:39:58+00:00,2022-06-01 00:35:27+00:00,"[4.28209, 51.89865]",BOTL,2,POINT (4.28209 51.89865)
8,2022-05-31 23:34:58+00:00,2022-06-01 01:20:40+00:00,"[4.14163, 51.96557]",3PET,4,POINT (4.14163 51.96557)
...,...,...,...,...,...,...
42214,2023-05-31 18:03:47+00:00,2023-05-31 20:05:16+00:00,"[4.12817, 51.96332]",7PET,24481,POINT (4.12817 51.96332)
42219,2023-05-31 21:01:36+00:00,2023-05-31 21:47:57+00:00,"[4.19908, 51.92873]",CKBRUG,24485,POINT (4.19908 51.92873)
42224,2023-05-31 21:17:18+00:00,2023-05-31 22:33:43+00:00,"[4.27635, 51.90111]",3PET,24489,POINT (4.27635 51.90111)
42228,2023-05-31 23:04:20+00:00,2023-05-31 23:44:50+00:00,"[4.1589, 51.94893]",CKVTTI,24492,POINT (4.15890 51.94893)
